In [44]:
import pandas as pd
import numpy as np

In [45]:
# AM import (both log and behavior)
behav_am_df = pd.read_excel(
    "C:/Users/HELIOS-300/Desktop/Data/am_behposture_onesheet.xlsx",
    engine="openpyxl"
)
log_df = pd.read_csv(
    "C:/Users/HELIOS-300/Desktop/Data/DO_LOG_final.csv",
    encoding="utf-8"
)

# Keep only State start rows (as you’re doing now)
behav_am_df = behav_am_df[behav_am_df["Event_Type"] == "State start"].copy()

# Convert timedelta columns back to time strings (HH:MM:SS format)
# Fixes the "0 days 00:00:00" display issue
for col in behav_am_df.columns:
    if pd.api.types.is_timedelta64_dtype(behav_am_df[col]):
        base_date = pd.Timestamp("1900-01-01")
        behav_am_df[col] = (base_date + behav_am_df[col]).dt.strftime("%H:%M:%S")

# ============================================================
# log_df: special AM setup / cleanup
# ============================================================

# Extract numeric id from log_df['id'] values like "AM02"
log_df["id"] = log_df["id"].astype(str).str.extract(r"AM(\d{2})", expand=False)
log_df["id"] = pd.to_numeric(log_df["id"], errors="coerce").astype("int64")

# Use obs (already formatted like DO2_a, DO2_b, DO1_a, etc.) as authoritative do label
log_df["do"] = log_df["obs"].astype(str).str.strip()

# Fix duplicated DO2 segments for id=11 and id=26 ONLY if they still appear as plain "DO2"
# (In your latest log, these are already DO2_a / DO2_b, so this will do nothing.)
for _id in [11, 26]:
    m = log_df["id"].eq(_id) & log_df["do"].eq("DO2")
    if m.sum() >= 2:
        tmp = log_df.loc[m].copy()
        tmp["_start_td"] = pd.to_timedelta(tmp["start_time"].astype(str).str.strip(), errors="coerce")
        idx_sorted = tmp.sort_values("_start_td", kind="mergesort").index

        if len(idx_sorted) >= 1:
            log_df.loc[idx_sorted[0], "do"] = "DO2_a"
        if len(idx_sorted) >= 2:
            log_df.loc[idx_sorted[1], "do"] = "DO2_b"
        if len(idx_sorted) > 2:
            for j in range(2, len(idx_sorted)):
                log_df.loc[idx_sorted[j], "do"] = f"DO2_b{j-1}"

# ============================================================
# behav_am_df: special AM setup / cleanup
# ============================================================

# id = 2 digits after AM in Observation
behav_am_df["id"] = (
    behav_am_df["Observation"]
    .str.extract(r"AM(\d{2})", expand=False)
    .astype("int64")
)

# do label (DO1, DO1_a, DO1_b, DO2, DO2_a, DO2_b, etc.)
behav_am_df["do"] = (
    behav_am_df["Observation"]
    .str.extract(r"(DO\d+(?:_[ab])?)", expand=False)
    .astype("string")
    .str.strip()
)

# normalize suffix case if it ever appears
behav_am_df["do"] = (
    behav_am_df["do"]
    .str.replace("_A", "_a", regex=False)
    .str.replace("_B", "_b", regex=False)
)

# For behav rows labeled DO2 but log has DO2_a / DO2_b (id=11, id=26),
# split using absolute timestamps (only affects rows where do == "DO2")
behav_am_df["_abs_dt_behav"] = pd.to_datetime(
    behav_am_df["Date_Time_Absolute_dmy_hmsf"],
    errors="coerce"
)

splits = {
    11: pd.Timestamp("2017-09-05 11:03:00"),
    26: pd.Timestamp("2018-02-24 17:13:00"),
}

for _id, cut in splits.items():
    m = (
        behav_am_df["id"].eq(_id)
        & behav_am_df["do"].eq("DO2")
        & behav_am_df["_abs_dt_behav"].notna()
    )
    behav_am_df.loc[m & (behav_am_df["_abs_dt_behav"] < cut), "do"] = "DO2_a"
    behav_am_df.loc[m & (behav_am_df["_abs_dt_behav"] >= cut), "do"] = "DO2_b"

# ============================================================
# NEW: add do_base for both dfs (strip only trailing _a/_b)
# This is what we’ll use for joins + the second-by-second backbone.
# ============================================================

log_df["do_base"] = (
    log_df["do"].astype(str).str.strip()
    .str.replace(r"_(a|b)$", "", regex=True)
)

behav_am_df["do_base"] = (
    behav_am_df["do"].astype(str).str.strip()
    .str.replace(r"_(a|b)$", "", regex=True)
)

# (keep _abs_dt_behav for QC; drop later if you want)
# behav_am_df.drop(columns=["_abs_dt_behav"], inplace=True, errors="ignore")

In [46]:
import pandas as pd
import numpy as np

# ============================================================
# CHUNK 1: behav_am_df -> sec_by_sec using ABSOLUTE time
#
# Rules:
#   - Group key = Observation (treat copyA/copyB as separate by default)
#   - Event start = Date_Time_Absolute_dmy_hmsf (authoritative)
#   - Coverage window per row = [start_sec, start_sec + ceil(Duration_sf)]  (inclusive)
#       (matches your example: 12:00:25 + 78 -> include through 12:01:43)
#   - Second grid per Observation goes from min(start_sec) to max(end_sec)
#   - For each second, pick the LAST-starting event that still covers that second
# ============================================================

df = behav_am_df.copy()

# Ensure datetime
df["Date_Time_Absolute_dmy_hmsf"] = pd.to_datetime(df["Date_Time_Absolute_dmy_hmsf"], errors="coerce")

# Drop rows that cannot anchor in time
df = df.dropna(subset=["Observation", "Date_Time_Absolute_dmy_hmsf"]).copy()

# Floor to seconds for alignment (keeps deterministic sec grid)
df["_start_dt_sec"] = df["Date_Time_Absolute_dmy_hmsf"].dt.floor("s")

# Duration in seconds (ceil), missing -> 0
df["_dur_s"] = pd.to_numeric(df["Duration_sf"], errors="coerce").fillna(0.0)
df["_dur_s_int"] = np.ceil(df["_dur_s"]).astype("int64")

# Inclusive end (IMPORTANT per your rule)
df["_end_dt_sec"] = df["_start_dt_sec"] + pd.to_timedelta(df["_dur_s_int"], unit="s")

# Sort for stable "latest start wins"
df = df.sort_values(["Observation", "_start_dt_sec", "Date_Time_Absolute_dmy_hmsf"], kind="mergesort")

out = []

# Choose columns to carry forward (everything except helpers)
helper_cols = {"_start_dt_sec", "_dur_s", "_dur_s_int", "_end_dt_sec"}
carry_cols = [c for c in df.columns if c not in helper_cols]

for obs, g in df.groupby("Observation", sort=False):
    g = g.copy()

    # Build the second grid using absolute dt
    start_dt = g["_start_dt_sec"].min()
    end_dt   = g["_end_dt_sec"].max()

    # If something is wildly wrong (should be <= ~2 hours), we still build but analysis will flag it
    grid = pd.date_range(start=start_dt, end=end_dt, freq="1s")

    # Prepare arrays for fast lookup
    starts = g["_start_dt_sec"].to_numpy()
    ends   = g["_end_dt_sec"].to_numpy()

    # For each grid second, find last event with start <= t
    # idx = rightmost start <= t
    tvals = grid.to_numpy()
    idx = np.searchsorted(starts, tvals, side="right") - 1

    # Build result frame with empty rows
    res = pd.DataFrame({"Observation": obs, "date_time_abs": grid})
    res["_sec"] = (res["date_time_abs"] - start_dt).dt.total_seconds().astype("int64")
    res["time_abs_hms"] = res["date_time_abs"].dt.strftime("%H:%M:%S")
    res["time_rel"] = pd.to_timedelta(res["_sec"], unit="s").astype(str).str.replace("0 days ", "", regex=False).str.zfill(8)

    # Mark covered seconds: idx>=0 and t <= end_of_that_event
    valid = (idx >= 0)
    valid &= (tvals <= ends[np.maximum(idx, 0)])

    # Attach event row data (latest-starting that still covers)
    # We only assign for covered seconds; uncovered stay NaN for event fields
    if valid.any():
        take_rows = g.iloc[idx[valid]][carry_cols].reset_index(drop=True)
        # Align columns onto res
        for c in take_rows.columns:
            if c in {"Observation"}:
                continue
            res.loc[valid, c] = take_rows[c].to_numpy()

    out.append(res)

sec_by_sec = pd.concat(out, ignore_index=True)

# remove helper-ish columns that might have been carried
sec_by_sec = sec_by_sec.drop(columns=[c for c in ["_dur_s", "_dur_s_int"] if c in sec_by_sec.columns], errors="ignore")

print("sec_by_sec shape:", sec_by_sec.shape)
print("unique Observation:", sec_by_sec["Observation"].nunique())
sec_by_sec.head()


sec_by_sec shape: (424022, 26)
unique Observation: 61


,Observation,date_time_abs,_sec,time_abs_hms,time_rel,Date_Time_Absolute_dmy_hmsf,Date_dmy,Time_Absolute_hms,Time_Absolute_f,Time_Relative_hmsf,...,Modifier_1,Modifier_2,Modifier_3,Modifier_4,Event_Type,Comment,id,do,_abs_dt_behav,do_base
0,AM01DO1_J_FINAL_R,2018-03-07 13:30:26,0,13:30:26,00:00:00,2018-03-07 13:30:26.511,2018-03-07 13:30:26.511,2018-03-07 13:30:26,511.0,00:00:00,...,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1,2018-03-07 13:30:26.511,DO1
1,AM01DO1_J_FINAL_R,2018-03-07 13:30:27,1,13:30:27,00:00:01,2018-03-07 13:30:26.511,2018-03-07 13:30:26.511,2018-03-07 13:30:26,511.0,00:00:00,...,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1,2018-03-07 13:30:26.511,DO1
2,AM01DO1_J_FINAL_R,2018-03-07 13:30:28,2,13:30:28,00:00:02,2018-03-07 13:30:26.511,2018-03-07 13:30:26.511,2018-03-07 13:30:26,511.0,00:00:00,...,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1,2018-03-07 13:30:26.511,DO1
3,AM01DO1_J_FINAL_R,2018-03-07 13:30:29,3,13:30:29,00:00:03,2018-03-07 13:30:26.511,2018-03-07 13:30:26.511,2018-03-07 13:30:26,511.0,00:00:00,...,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1,2018-03-07 13:30:26.511,DO1
4,AM01DO1_J_FINAL_R,2018-03-07 13:30:30,4,13:30:30,00:00:04,2018-03-07 13:30:26.511,2018-03-07 13:30:26.511,2018-03-07 13:30:26,511.0,00:00:00,...,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1,2018-03-07 13:30:26.511,DO1


In [47]:
# ------------------------------------------------------------
# Carry forward Behavior + Modifiers + Comment within Observation
# (ffill only; if the first value is NaN, it stays NaN)
# ------------------------------------------------------------

cols_to_ffill = ["Behavior", "Modifier_1", "Modifier_2", "Modifier_3", "Modifier_4", "Comment"]
cols_to_ffill = [c for c in cols_to_ffill if c in sec_by_sec.columns]

sec_by_sec = sec_by_sec.sort_values(["Observation", "date_time_abs"], kind="mergesort")
sec_by_sec[cols_to_ffill] = (
    sec_by_sec.groupby("Observation", sort=False)[cols_to_ffill].ffill()
)

# quick check: NaNs can still exist if the first event is NaN
sec_by_sec[cols_to_ffill].isna().sum()

Behavior           0
Modifier_1     30032
Modifier_2    357069
Modifier_3     46040
Modifier_4    330396
Comment       152803
dtype: int64

In [48]:
# ------------------------------------------------------------
# Keep only the required time columns and rename them
# ------------------------------------------------------------

# Columns to keep (if present)
keep_map = {
    "date_time_abs": "date_time",
    "time_abs_hms": "time",
    "time_rel": "rel_time",
    "Duration_sf": "duration",
}

# Drop other time-based columns if they exist
time_cols_to_drop = [
    "_sec",
    "Date_Time_Absolute_dmy_hmsf",
    "Date_dmy",
    "Time_Absolute_hms",
    "Time_Absolute_f",
    "Time_Relative_hmsf",
    "Time_Relative_hms",
    "Time_Relative_f",
    "Time_Relative_sf",
]

sec_by_sec = sec_by_sec.drop(columns=[c for c in time_cols_to_drop if c in sec_by_sec.columns])
sec_by_sec = sec_by_sec.rename(columns=keep_map)

# ------------------------------------------------------------
# Keep do_base only and rename to do_session
# ------------------------------------------------------------

if "do_base" in sec_by_sec.columns:
    sec_by_sec = sec_by_sec.drop(columns=["do"], errors="ignore")
    sec_by_sec = sec_by_sec.rename(columns={"do_base": "do_session"})

# ------------------------------------------------------------
# Drop QC-only absolute datetime column
# ------------------------------------------------------------

sec_by_sec = sec_by_sec.drop(columns=["_abs_dt_behav"], errors="ignore")

sec_by_sec.head()

,Observation,date_time,time,rel_time,duration,Event_Log,Behavior,Modifier_1,Modifier_2,Modifier_3,Modifier_4,Event_Type,Comment,id,do_session
0,AM01DO1_J_FINAL_R,2018-03-07 13:30:26,13:30:26,00:00:00,35.8486,Event log,LA- stand,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1
1,AM01DO1_J_FINAL_R,2018-03-07 13:30:27,13:30:27,00:00:01,35.8486,Event log,LA- stand,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1
2,AM01DO1_J_FINAL_R,2018-03-07 13:30:28,13:30:28,00:00:02,35.8486,Event log,LA- stand,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1
3,AM01DO1_J_FINAL_R,2018-03-07 13:30:29,13:30:29,00:00:03,35.8486,Event log,LA- stand,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1
4,AM01DO1_J_FINAL_R,2018-03-07 13:30:30,13:30:30,00:00:04,35.8486,Event log,LA- stand,No movement,NaN,NaN,NaN,State start,NaN,1.0,DO1


In [ ]:
# ------------------------------------------------------------
# Last row per Observation (shows final rel_time per session)
# ------------------------------------------------------------

sec_by_sec_last = (
    sec_by_sec.sort_values(["Observation", "date_time"], kind="mergesort")
    .groupby("Observation", sort=False)
    .tail(1)
)

sec_by_sec_last[["Observation", "rel_time", "date_time", "time", "duration"]].head(20)